# Wright-Fisher model

In [1]:
from typing import List

import numpy as np

In [2]:
def wright_fisher_fwd(
    N: int = 100,
    T: int = 50,
    nSim: int = 20,
    theta_f: List[float] = [0.5],
    theta_h: List[float] = [0.05],
    theta_z0: List[float] = [0.1],
    seed: float = 100,
    verbose: bool = False,
) -> (np.ndarray, np.ndarray, np.ndarray):
    """Forward simulation of the Wright-Fisher model.

    Args:
        N (int, optional): size of population. Defaults to 100.
        T (int, optional): time-points. Defaults to 50.
        nSim (int, optional): number of forward simulations. Defaults to 20.
        theta_f (List[float], optional): log relative fitness(es) of given variant(s). Defaults to a single variant with 0.5.
        theta_h (List[float], optional): mutation rate of variant(s). Defaults to single variant with mutation rate 0.05.
        theta_z0 (List[float], optional): initial probability of variant(s). Defaults to single variant with probability 0.1.
        seed (float, optional): random seed. Defaults to 100.
        verbose (bool, optional): increase printing verbosity. Defaults to False.

    Returns:
        Zs, Pis, log_Ps ((numpy.ndarray, numpy.ndarray, numpy.ndarray)): _description_

    Examples:
        To run with default paramaters

            >>> Zs, Pis, log_Ps = fisher_wright_fwd()

    Todo:
        * Upgrade nested List outputs to numpy.ndarray
    """

    Zs = [None] * nSim
    Pis = [None] * nSim  # np.empty((nSim))
    log_Ps = np.zeros((nSim))
    Dz = len(theta_f)

    np.random.seed(seed)

    for cSim in range(nSim):

        if verbose:
            print(f"\nsimulation: {cSim}")

        Z = np.zeros((T, N, Dz))
        Pi = np.zeros((T, N))
        for i in range(Dz):
            Z[0, :, i] = np.random.rand(1, N) < theta_z0[i]
            log_Ps[cSim] = np.sum(
                Z[0, :, i] * np.log(theta_z0[i])
                + (1 - Z[0, :, i]) * np.log(1 - theta_z0[i])
            )

        for t in range(1, T):
            fs = np.zeros((N))
            for i in range(Dz):
                fs = fs + Z[t - 1, :, i] * theta_f[i]

            fs = np.exp(fs)
            fs = fs / np.sum(fs)
            fs_ = np.cumsum(fs)

            for n in range(N):
                idx = np.argwhere(np.random.rand() <= fs_).flatten()
                idx = idx[0]
                Pi[t, n] = idx
                log_Ps[cSim] = log_Ps[cSim] + np.log(fs[idx])
                for i in range(Dz):
                    if np.random.rand() >= theta_h[i]:
                        Z[t, n, i] = Z[t - 1, idx, i]
                        log_Ps[cSim] = log_Ps[cSim] + np.log(1 - theta_h[i])
                    else:
                        Z[t, n, i] = 1 - Z[t - 1, idx, i]
                        log_Ps[cSim] = log_Ps[cSim] + np.log(theta_h[i])

        Zs[cSim] = Z
        Pis[cSim] = Pi

        if verbose:
            # [print(x.shape) for x in (Z, Pi, log_Ps)]
            # [print(x.shape) for x in (fs,)]
            # [print(np.array(x, dtype=object).shape) for x in (Zs, Pis)]
            for key, value in dict(
                [
                    ("Z", Z),
                    ("Pi", Pi),
                    ("log_Ps", log_Ps),
                    ("fs", fs),
                    ("Zs", Zs),
                    ("Pis", Pis),
                ]
            ).items():
                print(f"size {key}: {np.array(value, dtype=object).shape}")

    return Zs, Pis, log_Ps

In [3]:
# fwd simulations
nSim1 = 3
# bkw simulations
nSim2 = 5

# time-points
T = 4

# size of population
N = 5

# log relative fitness of variant
theta_f = [0.5]
# mutation rate
theta_h = [0.05]
# initial probability of variant
theta_z0 = [0.1]

# mixing param for bkw simulation
alpha = 0.85 * np.ones((1, T))
# verbosity
verbose = True
# for smoothing proposal dist
ep = 0

seed = 100

In [4]:
Zs, Pis, log_Ps = wright_fisher_fwd(
    N, T, nSim1, theta_f, theta_h, theta_z0, seed, verbose
);


simulation: 0
size Z: (4, 5, 1)
size Pi: (4, 5)
size log_Ps: (3,)
size fs: (5,)
size Zs: (3,)
size Pis: (3,)

simulation: 1
size Z: (4, 5, 1)
size Pi: (4, 5)
size log_Ps: (3,)
size fs: (5,)
size Zs: (3,)
size Pis: (3,)

simulation: 2
size Z: (4, 5, 1)
size Pi: (4, 5)
size log_Ps: (3,)
size fs: (5,)
size Zs: (3, 4, 5, 1)
size Pis: (3, 4, 5)


In [5]:
type(Zs)

list

In [6]:
assert np.array(Zs).shape == (nSim1, T, N, len(theta_f))

In [7]:
assert np.array(Pis).shape == (nSim1, T, N)

In [8]:
assert np.array(log_Ps).shape == (nSim1,)

In [9]:
for key, value in dict([("Zs", Zs), ("Pis", Pis), ("log_Ps", log_Ps)]).items():
    print(f"size {key}: {np.array(value).shape}")

size Zs: (3, 4, 5, 1)
size Pis: (3, 4, 5)
size log_Ps: (3,)
